In [142]:
import pandas as pd
import numpy as np

data = pd.read_csv('data.csv').astype(float)
display(data)

,Value
0,61.00970
1,61.02723
2,61.04366
3,61.05968
4,61.07617
5,61.09316
6,61.10946
7,61.12674
8,61.14310
9,61.15900


In [143]:
data = data.to_numpy()

In [144]:
def calculate_wave_length(data):
    """Calculate the wavelength of the laser light using the Michelson interferometer data.
    Args:
        data (numpy.ndarray): The data from the Michelson interferometer.
    Returns:
        float: The calculated wavelength of the laser light in nm.
    """
    wave_length = 2 / 50 * abs(sum(data[0:6]) - sum(data[6:12])) / 36 * 1e6
    return wave_length[0].round(4)

print(calculate_wave_length(data))

relative_error = np.abs(636 - 632.8) / 632.8 * 100
print(relative_error.round(2))

661.1222
0.51


In [145]:
N = 550  # 总条纹变化数
n_group = 50  # 每组条纹数
reference_wavelength = 632.8  # 单位：nm
instrument_error = 0.00005  # 仪器误差，单位：mm
t = 1.96
K_p = 1.96 

def process_data(d, k=6):
    """逐差法处理数据
    Args:
        d (np.array): 12个原始d值测量数据
        k (int): 分组数（默认6组）
    Returns:
        delta_d (np.array): 逐差法计算结果
    """
    # 将数据分为前后两组
    d1 = d[:k]
    d2 = d[k:]
    return d2 - d1

# 计算波长基础值
delta_d = process_data(data)
delta_d_mean = np.mean(delta_d)
wavelength = (2 / 50 * delta_d_mean * 1e6) / 6  # 转换为nm

# A类不确定度计算
s = np.std(delta_d, ddof=1)  # 贝塞尔公式计算样本标准差
uA = s / np.sqrt(len(delta_d))

# B类不确定度（仪器误差）
uB = instrument_error * np.sqrt(2)

# 合成不确定度
uc_d = np.sqrt(t * uA**2 + K_p * uB**2)

# 波长不确定度计算（误差传递）
uc_lambda = wavelength * np.sqrt((uc_d/delta_d_mean)**2 + (0.5/N)**2)

# 相对误差计算
relative_error = abs((wavelength - reference_wavelength)/reference_wavelength)*100

# 结果输出
print(f"逐差法计算平均Δd = {delta_d_mean:.4f} mm")
print(f"A类不确定度 uA = {uA*1e3:.2f} μm")
print(f"B类不确定度 uB = {uB*1e3:.3f} μm")
print(f"合成不确定度 uc(Δd) = {uc_d*1e3:.3f} μm")
print(f"计算波长 λ = {wavelength:.1f} ± {uc_lambda:.1f} nm")
print(f"相对误差 = {relative_error:.2f}%")

逐差法计算平均Δd = 0.0992 mm
A类不确定度 uA = 0.25 μm
B类不确定度 uB = 0.071 μm
合成不确定度 uc(Δd) = 0.360 μm
计算波长 λ = 661.1 ± 2.5 nm
相对误差 = 4.48%
